In [1]:
import pandas as pd
import pickle
from scipy.stats import poisson


In [2]:
dict_table = pickle.load(open("dict_groups", "rb"))
df_historical_data = pd.read_csv("clean_fifa_worldcup_historial_data.csv")
df_fixture = pd.read_csv("clean_fifa_worldcup_fixture.csv")

In [356]:
# Comprobamos los datos
# dict_table["Grupo A"]
#df_fixture

1- CALCULAR EL PODERIO DE UN EQUIPO

In [3]:
df_historical_data

,Pais(L),Pais(V),Year,Goles(Local),Goles(Visita),Goles_Totales
0,China,Alemania,1991,2,2,4
1,Estados Unidos,Francia,1991,1,0,1
2,Brasil,Estados Unidos,1991,3,0,3
3,Alemania,Francia,1991,1,2,3
4,China,Estados Unidos,1991,3,1,4
...,...,...,...,...,...,...
281,Alemania,Suecia,2019,1,2,3
282,Inglaterra,Estados Unidos,2019,1,2,3
283,Países Bajos,Suecia,2019,1,0,1
284,Inglaterra,Suecia,2019,1,2,3


In [4]:
#Dividir dataframe histórico en df_local y df_visita
df_local = df_historical_data[["Pais(L)", "Goles(Local)", "Goles(Visita)" ]]
df_visita = df_historical_data[["Pais(V)", "Goles(Local)", "Goles(Visita)" ]]

In [5]:
df_local

,Pais(L),Goles(Local),Goles(Visita)
0,China,2,2
1,Estados Unidos,1,0
2,Brasil,3,0
3,Alemania,1,2
4,China,3,1
...,...,...,...
281,Alemania,1,2
282,Inglaterra,1,2
283,Países Bajos,1,0
284,Inglaterra,1,2


In [6]:
#Renombrar columnas
df_local = df_local.rename(columns={"Pais(L)":"Equipo", "Goles(Local)":"GolesAnotados", "Goles(visita)":"GolesRecibidos"})
df_visita = df_visita.rename(columns={"Pais(V)":"Equipo", "Goles(Local)":"GolesRecibidos", "Goles(visita)":"GolesAnotados"})

In [ ]:
#Procedemos a concatenar df_local y df_visita, hacemos grupo por "Equipo" y calculamos el promedio
df_poderio_equipo = pd.concat([df_local, df_visita], ignore_index=True).groupby("Equipo").mean()
df_poderio_equipo
#Obtendremos el poderio de cada equipo

2- FUNCIÓN PREDICCION DE PUNTOS

In [ ]:
''' -> Los eventos se pueden contar
    -> La ocurrencia de eventos son independientes
    -> La tasa para que un evento ocurra, es constante
    -> Dos eventos no pueden ocurrir al mismo tiempo (un gol, no puede ocurrir en un musmo segundo)'''

In [9]:
def predict_points(local, visitante):
    if local in df_poderio_equipo.index and visitante in df_poderio_equipo.index:
        # goles anotados * goles recibidos
        lamb_local = df_poderio_equipo.at[local,"GolesAnotados"] * df_poderio_equipo.at[visitante,"GolesRecibidos"]
        lamb_visitante = df_poderio_equipo.at[visitante,"GolesAnotados"] * df_poderio_equipo.at[local,"GolesRecibidos"]
        prob_local, prob_visitante, prob_empate = 0, 0, 0
        for x in range(0,11): #numero de goles equipo local
            for y in range(0, 11): #numero de goles equipo visitante
                p = poisson.pmf(x, lamb_local) * poisson.pmf(y, lamb_visitante)
                if x == y:
                    prob_empate += p
                elif x > y:
                    prob_local += p
                else:
                    prob_visitante += p
        
        puntos_local = 3 * prob_local + prob_empate
        puntos_visitante = 3 * prob_visitante + prob_empate
        return (puntos_local, puntos_visitante)
    else:
        return (0, 0)

3- TESTEAMOS LA FUNCIÓN

In [10]:
#Testemos con partidos: Estados Unidos - Brasil, Países Bajos - España, Perú - Australia
predict_points("Estados Unidos","Brasil") 
predict_points("Países Bajos","España") 
predict_points("Perú","Australia")

(0, 0)

4-** PREDICIENDO EL MUNDIAL **

In [11]:
#Dividimos fixture en grupo, octavos, cuartos, ..
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_octavos = df_fixture[48:56].copy()
df_fixture_cuartos = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()


In [12]:
#Verficamos la division del fixture 
df_fixture_group_48
df_fixture_cuartos

,Local,Marcador,Visita,year
56,OF1,Partido 57,OF3,2023
57,OF2,Partido 58,OF4,2023
58,OF5,Partido 59,OF7,2023
59,OF6,Partido 60,OF8,2023


In [13]:
#Correr todos los partidos de la fase de grupo y actualizar las tablas de cada grupo
for grupos in dict_table:
    teams_in_group = dict_table[grupos]['Países'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['Local'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        Local, Visita = row['Local'], row['Visita']
        puntos_local, puntos_visitante = predict_points(Local,Visita)
        dict_table[grupos].loc[dict_table[grupos]['Países'] == Local, 'Pts.'] += puntos_local
        dict_table[grupos].loc[dict_table[grupos]['Países'] == Visita, 'Pts.'] += puntos_visitante

    dict_table[grupos] = dict_table[grupos].sort_values('Pts.', ascending=False).reset_index()
    dict_table[grupos] = dict_table[grupos][['Países', 'Pts.']]
    dict_table[grupos] = dict_table[grupos].round(0)

In [14]:
#Mostramos tabla actualizada [A-H]
dict_table["Grupo C"]

,Países,Pts.
0,España,5.0
1,Japón,4.0
2,Costa Rica,0.0
3,Zambia,0.0


5-**OCTAVOS DE FINAL **

In [15]:
df_fixture_octavos

,Local,Marcador,Visita,year
48,1A,Partido 49,2C,2023
49,1C,Partido 50,2A,2023
50,1E,Partido 51,2G,2023
51,1G,Partido 52,2E,2023
52,1D,Partido 54,2B,2023
53,1B,Partido 53,2D,2023
54,1H,Partido 56,2F,2023
55,1F,Partido 55,2H,2023


In [16]:
#Para cambiar los valores de las filas y poder ontener un "patrón", modificamos los datos
mapping = {
    "1A": "First Grupo A",
    "1B": "First Grupo B",
    "1C": "First Grupo C",
    "1D": "First Grupo D",
    "1E": "First Grupo E",
    "1F": "First Grupo F",
    "1G": "First Grupo G",
    "1H": "First Grupo H",
    "2A": "Second Grupo A",
    "2B": "Second Grupo B",
    "2C": "Second Grupo C",
    "2D": "Second Grupo D",
    "2E": "Second Grupo E",
    "2F": "Second Grupo F",
    "2G": "Second Grupo G",
    "2H": "Second Grupo H"
}

df_fixture_octavos["Local"] = df_fixture_octavos["Local"].map(mapping)
df_fixture_octavos["Visita"] = df_fixture_octavos["Visita"].map(mapping)

In [17]:
#Obtenemos el patrón 
df_fixture_octavos

,Local,Marcador,Visita,year
48,First Grupo A,Partido 49,Second Grupo C,2023
49,First Grupo C,Partido 50,Second Grupo A,2023
50,First Grupo E,Partido 51,Second Grupo G,2023
51,First Grupo G,Partido 52,Second Grupo E,2023
52,First Grupo D,Partido 54,Second Grupo B,2023
53,First Grupo B,Partido 53,Second Grupo D,2023
54,First Grupo H,Partido 56,Second Grupo F,2023
55,First Grupo F,Partido 55,Second Grupo H,2023


In [18]:
for grupos in dict_table:
    ganador_de_grupo = dict_table[grupos].loc[0, "Países"]
    segundo_de_grupo = dict_table[grupos].loc[1, "Países"]
    df_fixture_octavos.replace({f"First {grupos}": ganador_de_grupo, 
                                f"Second {grupos}": segundo_de_grupo}, inplace=True)
    df_fixture_octavos["Winner"] = "?"


In [19]:
df_fixture_octavos

,Local,Marcador,Visita,year,Winner
48,Suiza,Partido 49,Japón,2023,?
49,España,Partido 50,Noruega,2023,?
50,Estados Unidos,Partido 51,Italia,2023,?
51,Suecia,Partido 52,Países Bajos,2023,?
52,Inglaterra,Partido 54,Canadá,2023,?
53,Australia,Partido 53,China,2023,?
54,Alemania,Partido 56,Francia,2023,?
55,Brasil,Partido 55,Corea del Sur,2023,?


In [20]:
#Creamos función para obtener ganador
def get_winner(df_fixture_actualizado):
    for index, row in df_fixture_actualizado.iterrows():
        Local, Visita = row["Local"], row["Visita"]
        puntos_local, puntos_visitante = predict_points(Local, Visita)
        if puntos_local > puntos_visitante:
            Winner = Local
        else:
            Winner = Visita
        df_fixture_actualizado.loc[index, "Winner"] = Winner
    return df_fixture_actualizado

In [21]:
get_winner(df_fixture_octavos)

,Local,Marcador,Visita,year,Winner
48,Suiza,Partido 49,Japón,2023,Suiza
49,España,Partido 50,Noruega,2023,Noruega
50,Estados Unidos,Partido 51,Italia,2023,Estados Unidos
51,Suecia,Partido 52,Países Bajos,2023,Países Bajos
52,Inglaterra,Partido 54,Canadá,2023,Inglaterra
53,Australia,Partido 53,China,2023,China
54,Alemania,Partido 56,Francia,2023,Alemania
55,Brasil,Partido 55,Corea del Sur,2023,Brasil


6-** CUARTOS DE FINAL **

In [22]:
df_fixture_cuartos

,Local,Marcador,Visita,year
56,OF1,Partido 57,OF3,2023
57,OF2,Partido 58,OF4,2023
58,OF5,Partido 59,OF7,2023
59,OF6,Partido 60,OF8,2023


In [23]:
#Para cambiar los valores de las filas "OF"y poder ontener un "patrón", modificamos los datos
mapping = {
    "OF1": "Suiza",#"Winners Partido 49",
    "OF3": "Estados Unidos",#"Winners Partido 51",
    "OF2": "Noruega",#"Winners Partido 50",
    "OF4": "Países Bajos",#"Winners Partido 52",
    "OF5": "China",#"Winners Partido 53",
    "OF7": "Brasil",#"Winners Partido 55",
    "OF6": "Inglaterra",#"Winners Partido 54",
    "OF8": "Alemania"}#"Winners Partido 56"

df_fixture_cuartos["Local"] = df_fixture_cuartos["Local"].map(mapping)
df_fixture_cuartos["Visita"] = df_fixture_cuartos["Visita"].map(mapping)

In [24]:
#Verificamos
df_fixture_cuartos

,Local,Marcador,Visita,year
56,Suiza,Partido 57,Estados Unidos,2023
57,Noruega,Partido 58,Países Bajos,2023
58,China,Partido 59,Brasil,2023
59,Inglaterra,Partido 60,Alemania,2023


In [25]:
# Creamos la función tabla_actualizada 
def tabla_actualizada(df_fixture_round_1, df_fixture_round_2):
    for index, row in  df_fixture_round_1.iterrows():
        Winner = df_fixture_round_1.loc[index, "Winner"]
        Marcador = df_fixture_round_1.loc[index, "Marcador"]
        #df_fixture_round_2.replace({f"Winners {Marcador}":Winner}, inplace = True)
    df_fixture_round_2["Winner"] = "?"
    return df_fixture_round_2

In [26]:
tabla_actualizada(df_fixture_octavos, df_fixture_cuartos)

,Local,Marcador,Visita,year,Winner
56,Suiza,Partido 57,Estados Unidos,2023,?
57,Noruega,Partido 58,Países Bajos,2023,?
58,China,Partido 59,Brasil,2023,?
59,Inglaterra,Partido 60,Alemania,2023,?


In [27]:
'''Aplicamos get_winner para obtener ganadores de partido'''
get_winner(df_fixture_cuartos)

,Local,Marcador,Visita,year,Winner
56,Suiza,Partido 57,Estados Unidos,2023,Suiza
57,Noruega,Partido 58,Países Bajos,2023,Noruega
58,China,Partido 59,Brasil,2023,Brasil
59,Inglaterra,Partido 60,Alemania,2023,Alemania


7-** SEMIFINAL **

In [28]:
df_fixture_semi

,Local,Marcador,Visita,year
60,CFA,Partido 61,CFB,2023
61,CFC,Partido 62,CFD,2023


In [29]:
#Para cambiar los valores de las filas "OF"y poder ontener un "patrón", modificamos los datos
mapping = {
    "CFA": "Suiza",
    "CFB": "Noruega",
    "CFC": "Brasil",
    "CFD": "Alemania"}

df_fixture_semi["Local"] = df_fixture_semi["Local"].map(mapping)
df_fixture_semi["Visita"] = df_fixture_semi["Visita"].map(mapping)

In [30]:
df_fixture_semi

,Local,Marcador,Visita,year
60,Suiza,Partido 61,Noruega,2023
61,Brasil,Partido 62,Alemania,2023


In [31]:
tabla_actualizada(df_fixture_cuartos, df_fixture_semi)

,Local,Marcador,Visita,year,Winner
60,Suiza,Partido 61,Noruega,2023,?
61,Brasil,Partido 62,Alemania,2023,?


In [32]:
'''Aplicamos get_winner para obtener ganadores de partido'''
get_winner(df_fixture_semi)

,Local,Marcador,Visita,year,Winner
60,Suiza,Partido 61,Noruega,2023,Suiza
61,Brasil,Partido 62,Alemania,2023,Brasil


8-** FINAL COPA MUNDIAL **

In [33]:
df_fixture_final = df_fixture_final.drop(df_fixture_final.index[0])
mapping = {
    "WS1": "Suiza",
    "WS2": "Brasil"}

df_fixture_final["Local"] = df_fixture_final["Local"].map(mapping)
df_fixture_final["Visita"] = df_fixture_final["Visita"].map(mapping)

In [34]:
get_winner(df_fixture_final)

,Local,Marcador,Visita,year,Winner
63,Suiza,Partido 64,Brasil,2023,Suiza


Y la selección a ganar la Copa Mundial Femenina 2023 es Suiza :D